In [10]:
from modules import function as fc
import PySimpleGUI as sg
from rdkit import Chem
import pubchempy as pcp
import csv
import pandas as pd

sg.theme("DarkBlack1")


#官能基カウントのモジュールを空で実行し、官能基リストを作成
functional_names = fc.count_functional_groups().count.keys()

#記述子名のリスト
properties = ["Tag","ChemicalName","SMILES","MW (g/mol)","logP"]
#官能基名のリスト
functional_g = [f"{name} (mmol/g)" for name in functional_names]
#CSV出力用の２Dリスト
output_data = properties+functional_g


#過去データの取得、またはデータテーブルの初期化
try :
    #過去データの取得,DF化
    material_df = pd.read_csv("CSV/materials.csv")
    #Tagのみ
    past_tag = material_df["Tag"].tolist()
except:
    with open("CSV/materials.csv", 'w',newline="") as file:
        writer = csv.writer(file)
        writer.writerow(output_data)
    past_tag = pd.read_csv("CSV/materials.csv")["Tag"].tolist()


#全体のレイアウト
layout = [[sg.Text('Materials', font=('Constantia',20,"bold"))],
          [sg.Frame("Main",[[sg.Text("化合物名 / SMILES"),
                             sg.Input(size=(40, 1), 
                                      key="-INPUT-",
                                      text_color='black',
                                      background_color='honeydew'
                                     ),
                             sg.Text("タグ"),
                             sg.Input(size=(10, 1),key = "Tag" ,
                                      text_color='black',
                                      background_color='honeydew')],
                            [sg.Radio('CAS or NAME',default=True, key="-1-", group_id='0'),
                             sg.Radio('SMILES',default=False, key="-2-", group_id='0')],
                            [sg.Text("化学反応")],
                            [sg.Checkbox("ビニル重合",key="polym", default=False)],
                            [sg.Checkbox("カルボン酸の脱プロトン化",key="Acid_DH", default=False)],
                            [sg.Checkbox("アミンのプロトン化",key="Amine_H", default=False)], 
                            [sg.Button("実行",size=(10,1)),
                             sg.Button("記録",size=(10,1)),
                             sg.Button("削除",size=(10,1)),
                             sg.Button("読込み",size=(10,1)),
                             sg.Push(),
                             sg.Button("保存",size=(10,1))],
                            [sg.Image(key="-IMAGE-")],

                            [sg.Text("",size=(60, 1),key = "NAME")],
                            [sg.Text("",size=(60, 1),key = "SMILES")],
                            [sg.Text("",size=(60, 1),key = "MW")],
                            [sg.Text("",size=(60, 1),key = "LogP")],
                           ]),
           
           #MainFrameの終了 / 出力テーブルの開始
           sg.Column(layout =[[sg.Text("",size=(20,1),key=name),
                              sg.Text("", key=f"{name}_")] for name in functional_names]),
           sg.Table(headings =["　記録履歴　"],values = [[elem] for elem in past_tag],
                    key ="output_table",
                    size=(15,30))
           ]]


# モニターの解像度を取得
screen_width, screen_height = sg.Window.get_screen_size()

# ウィンドウのサイズをモニターの全画面に設定
window_size = (int(screen_width*4/5), screen_height)

# ウインドウの出現位置を指定
win_location = (0, 0)
window = sg.Window("MATERIALS", 
                   layout, 
                   size=window_size,
                   resizable=True,
                   location=win_location)


while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break
    if event == "実行":
        try:
                
            if values['-1-'] == True:
    
                input_val = values["-INPUT-"]
                results = pcp.get_compounds(input_val, 'name')
                compound = results[0]
                smiles = compound.canonical_smiles
                          
            elif values['-2-'] == True:
                
                input_val = values["-INPUT-"]
                iupac_name = "-" 
                smiles = input_val
    
            #化学反応
            
            if values["polym"] == True:
                smiles = fc.polym(smiles)
    
            if values["Acid_DH"] == True:
                smiles = fc.acid_takeHs(smiles)
    
            if values["Amine_H"] == True:
                smiles = fc.amine_addHs(smiles)                
                      
            #smilesから官能基個数のカウントモジュール実行
            fg = fc.count_functional_groups(smiles)
    
            # 画像をウィンドウ内のsg.Imageコンポーネントに更新
            window["-IMAGE-"].update("images/mol_structure.png")
            window["MW"].update(f"MW : {fg.mw} g/mol")
            window["LogP"].update(f"LogP : {fg.logP}")
            window["SMILES"].update(f"SMILES : {smiles}")
    
            for name in functional_names:
                window[name].update(f"{name} : {fg.count[name]}count")
                window[f"{name}_"].update(f"{round(1000*fg.count[name]/fg.mw,2)}mmol/g")
                
        except:
            print("something wrong!")

    #output.csvの中身のTagリストを取得

    if event == "記録":
        #追加データ行の作製
        tag = values["Tag"]
        chemical_name = values["-INPUT-"]
        new_row = [tag,chemical_name,smiles,fg.mw,fg.logP]+[1000*fg.count[name]/fg.mw for name in functional_names]
        #過去のTag名重複チェッカ－
        past_tag2 = material_df["Tag"].to_numpy()
        
        if tag in past_tag2:
            sg.popup("AlreadyExist",auto_close=True,auto_close_duration=1)
        else:
            new_row_df = pd.DataFrame([new_row],columns=output_data)
            material_df = pd.concat([material_df,new_row_df])
            #過去Tagデータの更新
            past_tag2 = material_df["Tag"].to_numpy()
            #Windowテーブルの更新
            window["output_table"].update([[elem] for elem in past_tag2])
            
    if event == "保存":                  
        material_df.to_csv("CSV/materials.csv",index=False)

        
    if event == "削除":
        delete_Tag = values["Tag"]
        material_df = material_df[material_df["Tag"] != delete_Tag]
        past_tag2 = material_df["Tag"].to_numpy()
        window["output_table"].update([[elem] for elem in past_tag2])

    if event == "読込み":
        Read_Name = values["Tag"]
        target_row =material_df[material_df["Tag"] == Read_Name]
        read_smiles = target_row["SMILES"].to_numpy()[0]
        window["-INPUT-"].update(read_smiles)
        window["-2-"].update(True)



window.close()

[00:47:28] SMILES Parse Error: syntax error while parsing: isobutyl
[00:47:28] SMILES Parse Error: Failed parsing SMILES 'isobutyl' for input: 'isobutyl'


something wrong!


In [4]:
new_row

['2EH',
 '*CC(*)C(=O)OCC(CC)CCCC',
 '*CC(*)C(=O)OCC(CC)CCCC',
 184.27899999999994,
 3.051400000000002,
 0.0,
 0.0,
 0.0,
 0.0,
 5.426554300815613,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 10.853108601631225,
 32.559325804893675,
 10.853108601631225,
 0.0,
 0.0,
 0.0,
 0.0]

In [8]:
output_data

[['Tag',
  'ChemicalName',
  'SMILES',
  'MW (g/mol)',
  'logP',
  'carboxyl (mmol/g)',
  'carboxylate (mmol/g)',
  'hydroxyl (mmol/g)',
  'ether (mmol/g)',
  'ester (mmol/g)',
  'epoxy (mmol/g)',
  'amide (mmol/g)',
  'cyano (mmol/g)',
  'amine (mmol/g)',
  'ammonium (mmol/g)',
  'phenyl (mmol/g)',
  'methyl (mmol/g)',
  'methylene (mmol/g)',
  'methyne (mmol/g)',
  'quat_carbon (mmol/g)',
  'vinyl (mmol/g)',
  'asetylene (mmol/g)',
  'siloxane (mmol/g)']]

In [6]:
new_row_df.columns

MultiIndex([(                 'Tag',),
            (        'ChemicalName',),
            (              'SMILES',),
            (          'MW (g/mol)',),
            (                'logP',),
            (   'carboxyl (mmol/g)',),
            ('carboxylate (mmol/g)',),
            (   'hydroxyl (mmol/g)',),
            (      'ether (mmol/g)',),
            (      'ester (mmol/g)',),
            (      'epoxy (mmol/g)',),
            (      'amide (mmol/g)',),
            (      'cyano (mmol/g)',),
            (      'amine (mmol/g)',),
            (   'ammonium (mmol/g)',),
            (     'phenyl (mmol/g)',),
            (     'methyl (mmol/g)',),
            (  'methylene (mmol/g)',),
            (    'methyne (mmol/g)',),
            ('quat_carbon (mmol/g)',),
            (      'vinyl (mmol/g)',),
            (  'asetylene (mmol/g)',),
            (   'siloxane (mmol/g)',)],
           )

In [7]:
pd.concat([material_df,new_row_df],axis=0)

,Tag,ChemicalName,SMILES,MW (g/mol),logP,carboxyl (mmol/g),carboxylate (mmol/g),hydroxyl (mmol/g),ether (mmol/g),ester (mmol/g),...,"(amine (mmol/g),)","(ammonium (mmol/g),)","(phenyl (mmol/g),)","(methyl (mmol/g),)","(methylene (mmol/g),)","(methyne (mmol/g),)","(quat_carbon (mmol/g),)","(vinyl (mmol/g),)","(asetylene (mmol/g),)","(siloxane (mmol/g),)"
0,CHCl3,chloroform,C(Cl)(Cl)Cl,119.378,1.9864,0.000000,0.0,0.000000,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PGME,propylene glycol methyl ether,CC(COC)O,90.122,0.0136,0.000000,0.0,11.096070,11.09607,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nBuOH,butanol,CCCCO,74.123,0.7788,0.000000,0.0,13.491089,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,iBuOH,isobutanol,CC(C)CO,74.123,0.6347,0.000000,0.0,13.491089,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IPA,isopropanol,CC(C)O,60.096,0.3871,0.000000,0.0,16.640043,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MMA,methyl methacrylate,*CC(*)(C)C(=O)OC,100.117,0.8549,0.000000,0.0,0.000000,0.00000,9.988314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,EMA,ethyl methacrylate,*CC(*)(C)C(=O)OCC,114.144,1.2450,0.000000,0.0,0.000000,0.00000,8.760863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,IBMA,isobutyl methacrylate,*CC(*)(C)C(=O)OCC(C)C,142.198,1.8811,0.000000,0.0,0.000000,0.00000,7.032448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2EHMA,2 ethylhexyl methacrylate,*CC(*)(C)C(=O)OCC(CC)CCCC,198.306,3.4415,0.000000,0.0,0.000000,0.00000,5.042712,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2EHA,2 ethylhexyl acrylate,*CC(*)C(=O)OCC(CC)CCCC,184.279,3.0514,0.000000,0.0,0.000000,0.00000,5.426554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
